### “Peak-Hour Congestion Analysis & Accident Pattern Detection for a Growing City”




📊 Required Visualizations
<br><br>
You must include at least 10 visualizations:
<br>
Line chart — hourly congestion trend
<br>
Heatmap — day vs hour congestion
<br>
Bar chart — congestion by road type
<br>
Boxplot — speed distribution by road type
<br>
Bar chart — accidents by severity
<br>
Time series plot — accidents across 24 hours
<br>
Pie chart — accident weather distribution
<br>
Scatter plot — GPS speed vs time
<br>
Cluster plot — accident or GPS clusters
<br>
Any additional chart you find useful<br><br>

In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [10]:
traffic_vol = pd.read_csv(r"data/Metro_Interstate_Traffic_Volume.csv.gz") 
traffic_vol.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


In [8]:
accidents = pd.read_csv(r"data/traffic_accidents.csv")
accidents.head()

,crash_date,traffic_control_device,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,crash_type,...,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,crash_hour,crash_day_of_week,crash_month
0,07/29/2023 01:00:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,13,7,7
1,08/13/2023 12:11:00 AM,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0,1,8
2,12/09/2021 10:30:00 AM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,10,5,12
3,08/09/2023 07:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,NONINCAPACITATING INJURY,5.0,0.0,0.0,5.0,0.0,0.0,19,4,8
4,08/19/2023 02:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,14,7,8


In [11]:
gps_traces = pd.read_csv(r"data/go_track_trackspoints.csv")
gps_traces.head()

,id,latitude,longitude,track_id,time,speed
0,1.0,-10.939341,-37.062742,1.0,9/13/2014 7:24,19.210586
1,2.0,-10.939341,-37.062742,1.0,9/13/2014 7:24,30.848229
2,3.0,-10.939324,-37.062765,1.0,9/13/2014 7:24,13.560101
3,4.0,-10.939211,-37.062843,1.0,9/13/2014 7:24,19.766679
4,5.0,-10.938939,-37.062879,1.0,9/13/2014 7:24,25.807401


1️⃣ Data Cleaning & Preparation
<br><br>
Remove missing sensor readings.
<br>
Convert timestamps properly.
<br>
Detect and remove impossible values (speed = 300 km/h, vehicle_count negative).
<br>
Create additional columns:
<br>
hour, day_of_week, is_weekend
<br>
congestion_level based on vehicle_count + avg_speed
<br><br>

In [13]:
traffic_vol.dropna()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
126,Columbus Day,273.080,0.00,0.0,20,Clouds,few clouds,2012-10-08 00:00:00,455
1123,Veterans Day,288.120,0.00,0.0,87,Clear,sky is clear,2012-11-12 00:00:00,1000
1370,Thanksgiving Day,278.540,0.00,0.0,20,Mist,mist,2012-11-22 00:00:00,919
2360,Christmas Day,264.400,0.00,0.0,90,Clouds,overcast clouds,2012-12-25 00:00:00,803
2559,New Years Day,263.490,0.00,0.0,58,Clouds,broken clouds,2013-01-01 00:00:00,1439
...,...,...,...,...,...,...,...,...,...
44441,Memorial Day,299.487,0.00,0.0,24,Clouds,few clouds,2018-05-28 00:00:00,1088
45547,Independence Day,297.550,0.00,0.0,1,Mist,mist,2018-07-04 00:00:00,1021
46936,State Fair,289.020,0.00,0.0,1,Clear,sky is clear,2018-08-23 00:00:00,596
47330,Labor Day,292.430,0.25,0.0,1,Rain,light rain,2018-09-03 00:00:00,962


In [16]:
traffic_vol['date_time'] = pd.to_datetime(traffic_vol['date_time'], format='%m/%d/%Y %H:%M' ,errors='coerce')
traffic_vol['date_time'].dtype

dtype('<M8[ns]')

In [19]:
traffic_vol[traffic_vol['traffic_volume'] > 0].dropna()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
126,Columbus Day,273.080,0.00,0.0,20,Clouds,few clouds,2012-10-08,455
1123,Veterans Day,288.120,0.00,0.0,87,Clear,sky is clear,2012-11-12,1000
1370,Thanksgiving Day,278.540,0.00,0.0,20,Mist,mist,2012-11-22,919
2360,Christmas Day,264.400,0.00,0.0,90,Clouds,overcast clouds,2012-12-25,803
2559,New Years Day,263.490,0.00,0.0,58,Clouds,broken clouds,2013-01-01,1439
...,...,...,...,...,...,...,...,...,...
44441,Memorial Day,299.487,0.00,0.0,24,Clouds,few clouds,2018-05-28,1088
45547,Independence Day,297.550,0.00,0.0,1,Mist,mist,2018-07-04,1021
46936,State Fair,289.020,0.00,0.0,1,Clear,sky is clear,2018-08-23,596
47330,Labor Day,292.430,0.25,0.0,1,Rain,light rain,2018-09-03,962


In [23]:
traffic_vol['hour'] = traffic_vol['date_time'].dt.hour
traffic_vol['day_of_week'] = traffic_vol['date_time'].dt.day_name()
traffic_vol['is_weekend'] = (traffic_vol['day_of_week']=="Saturday") | (traffic_vol['day_of_week']=="Sunday")
traffic_vol

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,hour,day_of_week,is_weekend
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,9,Tuesday,False
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,10,Tuesday,False
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,11,Tuesday,False
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,12,Tuesday,False
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,13,Tuesday,False
...,...,...,...,...,...,...,...,...,...,...,...,...
48199,NaN,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00,3543,19,Sunday,True
48200,NaN,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00,2781,20,Sunday,True
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159,21,Sunday,True
48202,NaN,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00,1450,22,Sunday,True


2️⃣ Analysis Tasks<br><br>
A. Peak Hour Congestion<br>

Find which hours and days experience the least traffic volume.
<br>
Check if traffic volume is worse on weekends or weekdays.
<br>
Compare road types:
<br>
highway vs main road vs residential
<br><br>


In [ ]:
least = traffic_vol.sort_values(by='traffic_volume').tail().index
for i in least:
    pd.DataFrame({
        'traffic volume' : least.value[i],
    })

AttributeError: module 'pandas' has no attribute 'DataFram'

In [ ]:
B. Accident Hotspot Analysis
<br><br>
Identify which areas/locations have the highest accident frequency.
<br>
Check accident severity distribution.
<br>
Find relationship between weather and accident severity.
<br>
Analyze time-based accident trends:
<br>
Do more accidents occur at night?
<br>
Are mornings safer?
<br><br>
C. Speed Pattern Analysis (GPS + Sensors)
<br><br>
Use GPS data to estimate “real speed trends” across the day.
<br>
Compare with sensor speeds—how accurate are sensors?
<br>
Find if speed drops near historical accident hotspots.
<br><br>
D. Clustering Task (Not Too Easy)
<br><br>
Use KMeans or DBSCAN to cluster:
<br>
accident locations OR
<br>
GPS coordinates (for common routes)
<br>
Find:
<br>
Dense route clusters
<br>
Congested road clusters
<br><br>